In [5]:
!pip install dm-tree graphviz
!pip install dm_haiku
!pip install chex optax rlax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 128 kB 4.3 MB/s 
     |████████████████████████████████| 272 kB 45.2 MB/s 


In [8]:
import os, sys
# sys.path = [os.path.abspath('drive/Othercomputers/grl')]
sys.path.append(os.path.abspath('/content/drive/Othercomputers/My MacBook Pro/grl'))
print(sys.path)

['/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/Othercomputers/My\\ MacBook\\ Pro/grl', '/content/drive/Othercomputers/My\\ MacBook\\ Pro/grl', '/content/drive/Othercomputers/My MacBook Pro/grl']


In [2]:
import os
!cd /content/drive/Othercomputers/My\ MacBook\ Pro/grl
os.chdir('/content/drive/Othercomputers/My MacBook Pro/grl')
os.getcwd()

'/content/drive/Othercomputers/My MacBook Pro/grl'

In [18]:
import jax.numpy as jnp
import haiku as hk

from core.typing import dict2AttrDict
from env.func import create_env
from algo.zero_mr.elements.model import create_model
from algo.zero_mr.elements.loss import create_loss
from algo.zero_mr.elements.trainer import create_trainer
from tools.yaml_op import load_config
from tools.display import pwc
config = load_config('algo/zero_mr/configs/magw_a2c')

def construct_fake_data(env_stats, aid):
    n = 2
    b = 32
    s = 4
    u = len(env_stats.aid2uids[aid])
    basic_shape = (n, b, s, u)
    shapes = env_stats.obs_shape[aid]
    dtypes = env_stats.obs_dtype[aid]
    action_dim = env_stats.action_dim[aid]
    data = {k: jnp.zeros((n, b, s+1, u, *v), dtypes[k]) 
        for k, v in shapes.items()}
    data = dict2AttrDict(data)
    data.global_state = data.obs
    data.action = jnp.zeros(basic_shape, jnp.int32)
    data.value = jnp.zeros(basic_shape, jnp.float32)
    data.reward = jnp.zeros(basic_shape, jnp.float32)
    data.discount = jnp.zeros(basic_shape, jnp.float32)
    data.reset = jnp.zeros(basic_shape, jnp.float32)
    data.mu_logprob = jnp.zeros(basic_shape, jnp.float32)
    data.mu = jnp.zeros((*basic_shape, action_dim), jnp.float32)
    
    return data

env = create_env(config.env)
model = create_model(config.model, env.stats())
loss = create_loss(config.loss, model)
trainer = create_trainer(config.trainer, env.stats(), loss)
data = construct_fake_data(env.stats(), 0)
pwc(hk.experimental.tabulate(trainer.raw_meta_train)(
    model.eta, model.theta, trainer.params, data), color='yellow')


/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."


+------------------------------------------+----------------------------------------------------------------------------------+-----------------+--------------------------------------------------+----------------------------------+---------------+---------------+
| Module                                   | Config                                                                           | Module params   | Input                                            | Output                           |   Param count |   Param bytes |
+==========================================+==================================================================================+=================+==================================================+==================================+===============+===============+
| meta_params (MetaParams)                 | MetaParams(                                                                      |                 | True                                             | {'entropy_c

In [ ]:
import graphviz
dot = hk.experimental.to_dot(trainer.raw_meta_train)(
    model.eta, model.theta, trainer.params, data)
graphviz.Source(dot)